# Homework 2 Part 2

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#### Q1 (2 points)

a. Implement a function that takes in the *train.csv* dataset and returns a cleaned dataset without outliers and NAs.

In [8]:
train = pd.read_csv('/Users/gracegupta/Desktop/grace_gupta_HW1/train.csv')

def preprocessing_pipeline(train):
    ## Your code goes here
    
    #get rid of NAs
    train = train.dropna()
    
    #get rid of outliers
    for X in train:
        mu = np.mean(train[X])
        std = np.std(train[X])
        normalized_data = (train[X]-mu)/std
        indexes = normalized_data < 3
        processed_train = train.loc[indexes, :]
    
    return processed_train

In [9]:
processed_train = preprocessing_pipeline(train)

In [10]:
processed_train.describe() #check that it's the same as part(a)

,Id,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1104.000000,1104.000000,1104.000000,1104.000000,1104.000000,1104.000000,1104.000000,1104.000000,1104.000000,1104.000000,1104.000000,1104.000000,1104.000000
mean,558.989130,8.327446,0.531943,0.270444,2.538451,0.087760,15.889946,46.425725,0.996784,3.312147,0.656821,10.416244,5.633152
std,323.789428,1.749430,0.180609,0.194891,1.286773,0.046592,10.358023,33.139474,0.001891,0.155899,0.166352,1.068044,0.805931
min,0.000000,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,278.750000,7.100000,0.400000,0.097500,1.900000,0.070000,7.000000,22.000000,0.995697,3.210000,0.550000,9.500000,5.000000
50%,557.500000,7.900000,0.520000,0.260000,2.200000,0.080000,14.000000,38.000000,0.996790,3.310000,0.620000,10.100000,6.000000
75%,840.250000,9.200000,0.640000,0.430000,2.600000,0.091000,22.000000,61.000000,0.997885,3.402500,0.730000,11.100000,6.000000
max,1118.000000,15.900000,1.580000,1.000000,15.400000,0.611000,68.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


b. Implement a function that takes in the R<sup>2</sup> of a regression and return the VIF value. 

In [11]:
def calculate_vif(r_squared):
    ## Your code goes here
    vif = 1/(1-r_squared)
    return vif

c. Implement a function that takes in preprocessed data and returns a dataframe that contains the VIF for each variable. The dataframe should contain two columns: variable names and their VIFs.

In [54]:
import statsmodels.api as sm

def generate_vif_dataframe(processed_train):
    ## Your code goes here
    data = []
    for X in processed_train:
        X_list = []
        X_list.append(X)
        i = processed_train.columns.get_loc(X)
        reg_list = [] #contains all other variables for regression against X
        for j in range(len(processed_train.columns)):
            if j != i :
                reg_list.append(processed_train.columns[j])
        model = sm.OLS(processed_train[X], sm.add_constant(processed_train[reg_list])).fit()
        rsq = model.rsquared
        vif = calculate_vif(rsq)
        X_list.append(vif)
        data.append(X_list)
        
    vif_dataframe = pd.DataFrame(data, columns = ['Variable name','VIF'])
                                
    return vif_dataframe

d. Use the functions that you developed from Q1 and generate a VIF dataframe for all the variables in the wine dataset. Discuss the VIF values you find and the effect of multicollinearity on regression models.

In [57]:
## Your code goes here
features_only = processed_train.loc[:, 'fixed acidity':'alcohol']
vif_dataframe = generate_vif_dataframe(features_only)
vif_dataframe

,Variable name,VIF
0,fixed acidity,7.718711
1,volatile acidity,1.782956
2,citric acid,3.159279
3,residual sugar,1.702938
4,chlorides,1.513004
5,free sulfur dioxide,1.971999
6,total sulfur dioxide,2.182210
7,density,6.147005
8,pH,3.388611
9,sulphates,1.415035


In [89]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [90]:
#TEST (check if VIFs are correct)
X = sm.add_constant(features_only)
pd.Series([variance_inflation_factor(X.values, i) 
               for i in range(X.shape[1])], 
              index=X.columns)

const                   1.650420e+06
fixed acidity           7.718711e+00
volatile acidity        1.782956e+00
citric acid             3.159279e+00
residual sugar          1.702938e+00
chlorides               1.513004e+00
free sulfur dioxide     1.971999e+00
total sulfur dioxide    2.182210e+00
density                 6.147005e+00
pH                      3.388611e+00
sulphates               1.415035e+00
alcohol                 3.022363e+00
dtype: float64

## Your discussion goes here
The variables fixed acidity and density have the highest VIF scores, suggesting that they are the most highly correlated with the other variables. Including multicollinear variables unnecessarily complicates the model and is misleading in terms of determining which factors actually affect wine quality.

#### Q2 (4 points)

a. Use the VIF dataframe you generated from Q1 and choose a subset of variables that does not have high multicollinearity.

In [250]:
## Your code goes here
#volatile acidity, residual sugar, chlorides, sulphates
X = processed_train[["volatile acidity", "residual sugar", "chlorides", "sulphates", "alcohol"]]
#X = processed_train[["volatile acidity", "chlorides", "sulphates", "total sulfur dioxide", "alcohol"]]
y = processed_train["quality"]

b. Fit a multiple linear regression model and print a summary of the regression result.

In [251]:
## Your code goes here
model_2b = sm.OLS(y, sm.add_constant(X)).fit()
model_2b.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                quality   R-squared:                       0.356
Model:                            OLS   Adj. R-squared:                  0.353
Method:                 Least Squares   F-statistic:                     121.2
Date:                Fri, 14 Feb 2020   Prob (F-statistic):          3.41e-102
Time:                        21:25:53   Log-Likelihood:                -1085.2
No. Observations:                1104   AIC:                             2182.
Df Residuals:                    1098   BIC:                             2212.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const                2.6965      0.235     11.493      0.000       2.236       3.157
volatile acidity    -1.1816      0.114    -10.357      0.000      -1.406      -0.958
residual sugar       0.0085      0.015      0.558      0.577      -0.021       0.039
chlorides           -1.8633      0.479     -3.887      0.000      -2.804      -0.923
sulphates            0.8997      0.135      6.682      0.000       0.635       1.164
alcohol              0.2992      0.019     15.491      0.000       0.261       0.337
==============================================================================
Omnibus:                        9.105   Durbin-Watson:                   2.024
Prob(Omnibus):                  0.011   Jarque-Bera (JB):               12.980
Skew:                          -0.007   Prob(JB):                      0.00152
Kurtosis:                       3.531   Cond. No.                         270.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

c. Use 5-fold cross validation to validate the mean squared error (MSE) of a multiple linear regression model fit to the subset of data. (HINT: You will need to implement a function to calculate the MSE. See this link for more details: https://en.wikipedia.org/wiki/Mean_squared_error).

In [195]:
## Your code goes here
def calc_mse(y, y_hat):
    n = np.size(y)
    sq_diff = (y-y_hat)**2
    mse = np.sum(sq_diff)/n
    return mse

In [248]:
split_train = processed_train[["volatile acidity", "residual sugar", "chlorides", "sulphates", "alcohol", "quality"]]
split_train = split_train.sample(frac=1) #shuffle the data
mult_dfs = np.array_split(split_train, 5) #split shuffled data into 5 dataframes

In [249]:
for i in range(len(mult_dfs)):
    test_df = mult_dfs[i]
    y_actual = test_df["quality"]
    X_test = test_df.loc[:, 'volatile acidity':'alcohol']
    #print(len(test_df.index)) = 221
    #create list of remaining dataframes to be concatenated into the training set
    train_list = []
    for j in range(len(mult_dfs)):
        if j != i:
            train_list.append(mult_dfs[j])
    train_df = pd.concat(train_list)
    #print(len(train_df.index))
    y_train = train_df["quality"]
    X_train = train_df.loc[:, 'volatile acidity':'alcohol']
    #fit model to training data
    model = sm.OLS(y_train, sm.add_constant(X_train)).fit()
    #use model to predict on test data
    y_pred = model.predict(sm.add_constant(X_test))
    #calculate mean squared error
    mse = calc_mse(y_actual, y_pred)
    print(mse)

0.3627226320987194
0.4601577305734633
0.41220873124529245
0.4178963336440911
0.46617627414197654


d. Make a prediction on the test set using the model you fit in 2b and submit to Kaggle.

In [197]:
## Your code goes here
test = pd.read_csv('/Users/gracegupta/Desktop/grace_gupta_HW1/test.csv')
test_X = test[["volatile acidity", "residual sugar", "chlorides", "sulphates", "alcohol"]]
#test_y = test["quality"]
pred = model_2b.predict(sm.add_constant(test_X))

sample_submission = pd.read_csv('/Users/gracegupta/Downloads/sample_submission.csv')
sample_submission.loc[:, 'Predicted'] = pred
sample_submission.to_csv('hw2_submission.csv', header=True, index=False)
sample_submission.head()

,Id,Predicted
0,0,5.977194
1,1,5.329781
2,2,5.314075
3,3,5.452076
4,4,5.427620


e. Compare the Kaggle score and the cross-validation result. Is the cross-validation result a good representation of the prediction accuracy on the test data?

In [ ]:
## Your discussion here

f. Discuss the difference between goodness of fit on the training set, on the validation set, and on the test set.

In [ ]:
## Your discussion here